<a href="https://colab.research.google.com/github/Falconwatch/llm_course/blob/main/HW1/%D0%94%D0%971.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Описание ДЗ1.

На основе семинара 1 предложите 2 метода улучшения построения эмбеддингов вопросов на основе word vectors.

За задание можно получить максимум 10 баллов.

За каждый метод можно получить максимум 5 баллов.
Разбалловка:
*   Воспроизводимость и читабельность кода - 1 балл.
*   Корректность метода - 1 балл.
*   Описание метода в техническом отчете - 2 балла.
*   Иновационность - 1 балл.


# 1. Информация о сабмите

Щербаков Игорь Андреевич

# 2. Технический отчет

***Введите сюда** подробное описание предложенных методов и экспериментов, с помощью которых вы пришли именно к выбору этих методов. НЕ вставляйте код в эту часть. Описание должно состоять минимум из 2-4 абзацев и содержать следующее: тип модели, параметры, как вы выбрали параметры, какие дальнейшие модификации готовых решений и т.д. вы использовали. Сюда можно включить, например, некоторые хитрости вашей предварительной обработки, описание моделей и мотивацию их использования, описание деталей процесса обучения. Если нужно, вставьте сюда графики, математические формулы.*

*Балл за «инновационность» будет присваиваться на основе содержания этой части. Если ваше отличие от бейзлайна это просто почистить тексты от стоп-слов или поменять одну базовую модель для построения word embeddings на другую, этот балл будет 0. Пробуйте разные подходы, модели, экспериментируйте с предварительной обработкой, параметрами и т. д. Можно переопределить уже существующий подход. Это нормально, что некоторые из ваших экспериментов не сработали так, как вы ожидали. Покажите нам, что вы проявили творческий подход и провели несколько экспериментов.*

<h2>Возможные шаги</h2>
Какие шаги были бы возможны:
<ul> 1. Предобработка данных х</ul>
<ul> 2. Подбор модели</ul>
<ul> 3. Подбор параметров модели</ul>
<ul> 4. Эксперименты с эмбедингами слов (добавление доп размерностей с новой информацией)</ul>
<ul> 5. Изменеиение способа получения представления фразы на основе представлений слов</ul>
<ul> 6. Модификация алгоритмов поиска похожих: повышение качества и скорости</ul>


Подбор разных моделей и их параметров, кажется весьма обыденным, поэтому в этом ноутбуке рассмотрим следующщие подходы: предобработку данных, дополнительные размерности в эмбединг слова, представление фразы на основе слов.

<h2>Что попробовал</h2>
<h3>Предобработка</h3>
В качестве дополнитльной предобработки были испоьзованы:
<li> Стемминг алгоритмом Snowball - ок, полезно убирать лишние формы слов, часто опечатки в конце
<li> Удаление стоп слов - отказался от него в пользу взвешивания
<li> Формировать вектор предложения как взвешенную сумму векторов слов. В качестве весов - значения idf. Идейно подход заменяет выкидывание стоп-слов, занижая вес сильно распространённых слов

<h3>Эксперименты с моделью</h3>
<li>1
<li>2



<h3>Поиск похожих</h3>
Алгоритм поиска похожих топиков на основе косинусного расстояния был заменнён на реализацию в FAISS, что позволило многократно ускорить поиск и повысить его релевантность.



# 3. *Code*

*Введите сюда весь код, использованный для получения результатов. Добавьте несколько комментариев и подразделов для навигации по вашему решению.*

*В этой части вам предстоит самостоятельно разработать решение задачи и предоставить воспроизводимый код:*
- *Использование Python 3;*
- *Содержит код для установки всех зависимостей;*
- *Содержит код для загрузки всех используемых наборов данных*;
- *Содержит код для воспроизведения ваших результатов (другими словами, если проверяющий загрузит ваш блокнот, он сможет выполнить код по ячейкам и получить результаты эксперимента, как описано в разделе методологии)*.


*В результате ваш код будет оценен по следующим критериям:*
- ***Читаемость**: ваш код должен быть хорошо структурирован, желательно с указанием частей вашего подхода (предварительная обработка, обучение модели, тестирование модели и т. д.).*
- ***Воспроизводимость**: ваш код должен воспроизводиться без ошибок в режиме «Выполнить все» (получение экспериментальной части).*


## 3.1 Импорт библиотек

In [1]:
#Установка бибилотек в окружение
#!pip install -q -r requirements.txt
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 28.9 MB/s eta 0:00:00


In [18]:
import numpy as np
from nltk.tokenize import WordPunctTokenizer
import gensim.downloader as api
from nltk.stem.snowball import SnowballStemmer
import nltk
from nltk.corpus import stopwords
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict

## 3.2 Загрузка данных

In [3]:
# download the data:

#!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
#!wget https://yadi.sk/i/BPQrUu1NaTduEw -O ./quora.txt

!wget https://raw.githubusercontent.com/Falconwatch/llm_course/main/HW1/quora.txt -O ./quora.txt



--2024-06-05 17:26:42--  https://raw.githubusercontent.com/Falconwatch/llm_course/main/HW1/quora.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33813903 (32M) [text/plain]
Saving to: ‘./quora.txt’

./quora.txt         100%[===================>]  32.25M   177MB/s    in 0.2s    

2024-06-05 17:26:43 (177 MB/s) - ‘./quora.txt’ saved [33813903/33813903]



In [4]:
data = np.array(list(open("./quora.txt", encoding="utf-8")))

## Предобработка данных

In [7]:
nltk.download("stopwords")
STOP_WORDS = set(stopwords.words('english'))
TOKENIZER = WordPunctTokenizer()
STEMMER = SnowballStemmer("english")

In [65]:
import string
def preprocess_phrase(phrase):
  lower_phrase = phrase.lower()
  lower_phrase = lower_phrase.translate(str.maketrans('', '', string.punctuation))

  tokens = TOKENIZER.tokenize(lower_phrase)
  stemmed_tokens = [STEMMER.stem(t) for t in tokens]

  return stemmed_tokens

preprocess_phrase("Hi there!")

['hi', 'there']

In [69]:
preprocessed_data = [preprocess_phrase(d) for d in data[:]]

In [70]:
#Формируем глобальный словарь частот
vectorizer = TfidfVectorizer()

data_for_tfidf = [" ".join(ws) for ws in preprocessed_data]
vectorizer.fit_transform(data_for_tfidf)

words = vectorizer.get_feature_names_out()
idfs = vectorizer.idf_
idfs_normalised = idfs/np.max(idfs)

WORDS_IDFS = defaultdict(lambda: 1.0, {w:i for w,i in zip(words, idfs_normalised)})

print(WORDS_IDFS["what"])
print(WORDS_IDFS["is"])
print(WORDS_IDFS["dog"])
print(WORDS_IDFS["?"])


0.14412748635454461
0.1572065412341149
0.48880240698138905
1.0


In [71]:
def process_phrase_and_get_idf(phrase):
  preprocessed_phrase = preprocess_phrase(phrase)
  phrase_idfs = [WORDS_IDFS[word] for word in preprocessed_phrase]
  phrase_idfs = phrase_idfs/np.sum(phrase_idfs)
  return preprocessed_phrase, np.array(phrase_idfs)

process_phrase_and_get_idf("what is dog?")


(['what', 'is', 'dog'], array([0.18240835, 0.19896126, 0.61863039]))

## 3.3. Модель

In [115]:
GLOVE_MODEL = api.load('glove-twitter-100')
GLOVE_WORDS_IN_DICT = MODEL.key_to_index.keys()


from gensim.models import FastText

FT_MODEL = FastText(vector_size=128, window=3, min_count=1)  # instantiate
FT_MODEL.build_vocab(corpus_iterable=preprocessed_data)
FT_MODEL.train(corpus_iterable=preprocessed_data, total_examples=len(preprocessed_data), epochs=3)
FT_WORDS_IN_DICT = model.wv.key_to_index.keys()


In [129]:
def get_phrase_embedding(phrase,
                         models:list,
                         words_dicts:list):
  full_vector_size = 0
  for model in models:
    full_vector_size += model.vector_size

  empty_vector = np.zeros([full_vector_size], dtype='float32')
  word_vectors = []

  phrase_words, words_weights = process_phrase_and_get_idf(phrase)

  #перебираю слова в фразе
  for word in phrase_words:
    #сюда буду класть эмбединги по слову
    one_word_vectors = list()
    #перебираю доступные модели
    for i in range(len(models)):
      model = models[i]
      words_in_dict = words_dicts[i]

      vector_store = model.wv if type(model) is FastText else model

      if type(model) is FastText:
        vector_store = model.wv
      else:
        vector_store = model

      if word in words_in_dict:
        one_word_vectors.append(vector_store.get_vector(word))
      else:
        one_word_vectors.append(np.zeros([model.vector_size], dtype='float32'))
    one_word_vector = np.concatenate(one_word_vectors)
    word_vectors.append(one_word_vector)

  phrase_vector = np.dot(words_weights, word_vectors)
  return phrase_vector

my_models = [GLOVE_MODEL, FT_MODEL]
my_words_dicts = [GLOVE_WORDS_IN_DICT, FT_WORDS_IN_DICT]

t1 = get_phrase_embedding("What is dog?", my_models, my_words_dicts)
#t2 = get_phrase_embedding("What is dog?", GLOVE_MODEL, GLOVE_WORDS_IN_DICT)

In [132]:
# compute vector embedding for all lines in data
data_vectors = np.array([ get_phrase_embedding(l, my_models, my_words_dicts) for l in data[:]])

In [133]:
class MyDb:
    def __init__(self, sentences, sentence_embedings):
        self._dim = len(sentence_embedings[0])
        self._faiss = faiss.IndexFlatL2(self._dim )
        self._faiss.add(sentence_embedings)
        self._sentences = sentences

    def get_similar(self, query_embeddings, k=5):
        result = []
        if (len(query_embeddings) == 0) or (len(query_embeddings[0])==0):
            raise Exception("query_embeddings должно быть списком векторов-запросов для поиска похожих")
        d, i = self._faiss.search(x= query_embeddings, k = k)

        for r in i:
            r_sent = self._sentences[i]
            result.append(r_sent)
        return result

In [134]:
MDB = MyDb(data, data_vectors)

## 3.4. Применение модели

In [140]:
def find_nearest(query, my_models, my_words_dicts, k=10):
    emb = get_phrase_embedding(query, my_models, my_words_dicts)
    result = MDB.get_similar(np.array([emb]), k)
    return result

In [141]:
find_nearest("How to eat potatoes", my_models, my_words_dicts)

[array([['How can tortoises eat tomatoes?\n',
         'Can you eat potatoes raw?\n',
         'How dangerous is it to eat raw eggs?\n',
         'How safe is it to eat raw hotdogs?\n',
         'How do you eat raw egg and salad in Japan?\n',
         'How safe is it to eat raw eggs?\n',
         'Can you eat fish and eggs together?\n',
         'How safe is it to eat raw steak?\n',
         'What can eating raw meat lead to?\n',
         'How do you eat coconut crabs?\n']], dtype='<U1170')]

In [143]:
find_nearest("Are all snakes venomous?", my_models, my_words_dicts)

[array([['Are all spiders venomous?\n',
         'How dangerous are corn snakes, and are they venomous?\n',
         'Are all insects omnivorous/herbivorous?\n',
         'Are all types of flesh edible?\n',
         "What are all of Henry Rollins's tattoos?\n",
         'Are there mutant snakes?\n', 'Do all cobras spit venom?\n',
         'Why are some spiders venomous?\n',
         'Are all-white pit bulls rare?\n',
         'What are the venomous spiders of Minnesota?\n']], dtype='<U1170')]

# FastText

(43868568, 61294640)

In [105]:
model.vector_size

128

In [110]:
type(model) is FastText

True

In [107]:
model.wv.get_vector("hello")

array([ 0.6862627 ,  1.1529286 , -0.03999156,  0.41668805, -0.08775536,
        0.4633573 , -0.93307227,  0.03430814,  1.1212194 ,  0.15501061,
        0.99429667, -1.0509496 , -0.52912676, -0.16949584, -0.27724844,
        1.0890815 , -0.6406821 ,  0.3036683 , -0.23967867,  1.5083961 ,
       -1.0448737 ,  1.1400263 ,  1.0975782 ,  0.5451354 ,  0.7920713 ,
        0.30796623, -0.7411187 ,  0.2733937 , -0.362613  ,  0.304511  ,
        0.71818846, -1.116722  ,  0.07348555,  0.9912328 ,  0.05032878,
       -0.56667906,  0.7744326 , -0.07837544, -0.6709083 ,  0.08309229,
       -0.5199062 , -0.15987472, -0.9971523 , -0.78810054,  0.21179396,
        0.40839264,  0.74007696, -0.12652978,  0.09335849,  0.41339588,
       -0.02734203, -1.0381933 ,  0.44872716,  0.14822537,  1.1556053 ,
        0.3851937 , -0.12182264,  0.7659448 , -0.9767002 , -0.6001767 ,
        0.826352  ,  1.0730821 , -0.80103076,  0.4354746 ,  0.44758016,
       -0.4289955 , -0.85899514,  0.9181729 ,  0.9911824 ,  1.39

In [104]:
model.wv.word_vec("hello")

<ipython-input-104-3c2124849c76>:1: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  model.wv.word_vec("hello")


array([ 0.6862627 ,  1.1529286 , -0.03999156,  0.41668805, -0.08775536,
        0.4633573 , -0.93307227,  0.03430814,  1.1212194 ,  0.15501061,
        0.99429667, -1.0509496 , -0.52912676, -0.16949584, -0.27724844,
        1.0890815 , -0.6406821 ,  0.3036683 , -0.23967867,  1.5083961 ,
       -1.0448737 ,  1.1400263 ,  1.0975782 ,  0.5451354 ,  0.7920713 ,
        0.30796623, -0.7411187 ,  0.2733937 , -0.362613  ,  0.304511  ,
        0.71818846, -1.116722  ,  0.07348555,  0.9912328 ,  0.05032878,
       -0.56667906,  0.7744326 , -0.07837544, -0.6709083 ,  0.08309229,
       -0.5199062 , -0.15987472, -0.9971523 , -0.78810054,  0.21179396,
        0.40839264,  0.74007696, -0.12652978,  0.09335849,  0.41339588,
       -0.02734203, -1.0381933 ,  0.44872716,  0.14822537,  1.1556053 ,
        0.3851937 , -0.12182264,  0.7659448 , -0.9767002 , -0.6001767 ,
        0.826352  ,  1.0730821 , -0.80103076,  0.4354746 ,  0.44758016,
       -0.4289955 , -0.85899514,  0.9181729 ,  0.9911824 ,  1.39